In [ ]:
import numpy as np
from gpcam.gp_optimizer import GPOptimizer
import matplotlib.pyplot as plt
from numpy.random import default_rng

import plotly.graph_objects as go
import numpy as np

In [ ]:
def plot(x,y,z,data = None):
    fig = go.Figure()
    fig.add_trace(go.Surface(x = x, y = y,z=z))
    if data is not None: 
        fig.add_trace(go.Scatter3d(x=data[:,0], y=data[:,1], z=data[:,2],
                                   mode='markers'))

    fig.update_layout(title='Posterior Mean', autosize=True,
                  width=800, height=800,
                  margin=dict(l=65, r=50, b=65, t=90))

    fig.show()


In [ ]:
a = np.load("/Users/davide/Downloads/us_topo.npy")
rng = default_rng()
ind = rng.choice(len(a)-1, size=50, replace=False)
points = a[ind,0:2]
values = a[ind,2:3]
print("x_min ", np.min(points[:,0])," x_max ",np.max(points[:,0]))
print("y_min ", np.min(points[:,1])," y_max ",np.max(points[:,1]))
print("length of data set: ", len(points))

index_set_bounds = np.array([[0,99],[0,248]])
hyperparameter_bounds = np.array([[0.001,1e9],[1,1000],[1,1000]])
hps_guess = np.array([4.71907062e+06, 4.07439017e+02, 3.59068120e+02])

In [ ]:
plt.figure()
plt.imshow(a[:,2].reshape(100,250))


In [ ]:
gp = GPOptimizer(2, index_set_bounds)
gp.tell(points,values)
gp.init_gp(hps_guess)
gp.train_gp(hyperparameter_bounds,pop_size = 20,tolerance = 1e-6,max_iter = 2)

In [ ]:
x_pred = np.empty((10000,2))
counter = 0
x = np.linspace(0,99,100)
y = np.linspace(0,248,100)

for i in x:
 for j in y:
   x_pred[counter] = np.array([i,j])
   counter += 1


In [ ]:
res1 = gp.posterior_mean(x_pred)

In [ ]:
res2 = gp.posterior_covariance(x_pred)
# #res3 = gp.gp.shannon_information_gain(x_pred)
# X,Y = np.meshgrid(x,y)

# PM = np.reshape(res1["f(x)"],(100,250))
# PV = np.reshape(res2["v(x)"],(100,250))
# # plot(X,Y,PM)
# # plot(X,Y,PV)


In [ ]:
X,Y = np.meshgrid(x,y)

PM = np.reshape(res1["f(x)"],(100,100))
PV = np.reshape(res2["v(x)"],(100,100))
# plot(X,Y,PM)
# plot(X,Y,PV)


In [ ]:
plot(X,Y,PM)
plot(X,Y,PV)

In [ ]:
from dask.distributed import Client, LocalCluster
cluster = LocalCluster()  # Launches a scheduler and workers locally
client = Client(cluster)  #

In [ ]:
# cluster.close()

In [ ]:

next = gp.ask(position = None, n = 5, acquisition_function = "variance", bounds = None,
             method = "hgdl", pop_size = 50, max_iter = 20,
             tol = 10e-6, dask_client = client)
print(next)

In [ ]:
next['x'][:,1]

In [ ]:
fig,ax=plt.subplots(2,1,figsize=(8,4))
ax[0].imshow(PM)
ax[1].imshow(a[:,2].reshape(100,250))

# plt.scatter(next['x'][:,1]/2.48,next['x'][:,0]/.99)

In [ ]:
res2

In [ ]:
res1['x'][199]

# Trying to predict experiment point

In [ ]:
import numpy as np
from gpcam.gp_optimizer import GPOptimizer
import matplotlib.pyplot as plt
from numpy.random import default_rng

import plotly.graph_objects as go
import numpy as np

In [ ]:
newpoint = find_next(points,values,gp)
newval = measure_next(a,newpoint)
values = np.append(values,newval)
points = np.append(points,np.array(newpoint)[None,:],axis=0)

print(values.shape,points.shape)
print(newpoint,newval)

def init_gp(points,values,index_set_bounds,hyperparameter_bounds,hps_guess):
    gp = GPOptimizer(2, index_set_bounds)
    gp.tell(points,values)
    gp.init_gp(hps_guess)
    gp.train_gp(hyperparameter_bounds,pop_size = 20,tolerance = 1e-6,max_iter = 2)
    return gp


def find_next(points,values,gp):
    gp.tell(points,values)
    gp.train_gp(hyperparameter_bounds,pop_size = 20,tolerance = 1e-6,max_iter = 2)
    new = gp.ask(position = None, n = 1, acquisition_function = "shannon_ig", bounds = None,
        method = "global", pop_size = 20, max_iter = 20, tol = 10e-6, x0 = None, dask_client = False)
    x,y = np.round(new['x'][0]).astype(int)
    return x,y

def measure_next(a,new):
    newval = a[new[0]*250+new[1]]
    assert np.allclose(new,newval[:2]),"coordinates don't match"
    # points.append(new)
    # values.append(a[new[0]*250+new[1]])
    return newval[2]
    # new,a[new[0]*250+new[1]][2]



In [ ]:
%%time
a = np.load("/Users/davide/Downloads/us_topo.npy")
rng = default_rng()
ind = rng.choice(len(a)-1, size=5, replace=False)
points = a[ind,0:2]
values = a[ind,2:3]
print("x_min ", np.min(points[:,0])," x_max ",np.max(points[:,0]))
print("y_min ", np.min(points[:,1])," y_max ",np.max(points[:,1]))
print("length of data set: ", len(points))

index_set_bounds = np.array([[0,99],[0,248]])
hyperparameter_bounds = np.array([[0.001,1e9],[1,1000],[1,1000]])
hps_guess = np.array([4.71907062e+06, 4.07439017e+02, 3.59068120e+02])

gp = init_gp(points,values,index_set_bounds,hyperparameter_bounds,hps_guess)
# gp = GPOptimizer(2, index_set_bounds)
# gp.tell(points,values)
# gp.init_gp(hps_guess)
# gp.train_gp(hyperparameter_bounds,pop_size = 20,tolerance = 1e-6,max_iter = 2)


# new = gp.ask(position = None, n = 1, acquisition_function = "covariance", bounds = None,
#      method = "global", pop_size = 20, max_iter = 20, tol = 10e-6, x0 = None, dask_client = False)

In [ ]:
x_pred = np.empty((25000,2))
counter = 0
x = np.linspace(0,99,100)
y = np.linspace(0,248,250)

for i in x:
 for j in y:
   x_pred[counter] = np.array([i,j])
   counter += 1

res1 = gp.posterior_mean(x_pred)
res2 = gp.posterior_covariance(x_pred)


X,Y = np.meshgrid(x,y)

PM = np.reshape(res1["f(x)"],(100,250))
PV = np.reshape(res2["v(x)"],(100,250))


fig,ax=plt.subplots(3,1,figsize=(8,6))
ax[0].imshow(PV)
ax[1].imshow(PM)
ax[2].imshow(a[:,2].reshape(100,250))
for aa in ax:
    aa.scatter(points[:-1,1],points[:-1,0],s=1,color='orange')
    aa.scatter(points[-1,1],points[-1,0],s=2,color='red')
    

In [ ]:
newpoint = find_next(points,values,gp)
newval = measure_next(a,newpoint)
values = np.append(values,newval)
points = np.append(points,np.array(newpoint)[None,:],axis=0)

print(values.shape,points.shape)
print(newpoint,newval)



In [ ]:
from tqdm.auto import tqdm
for i in tqdm(range(100)):
    newpoint = find_next(points,values,gp)
    newval = measure_next(a,newpoint)
    values = np.append(values,newval)
    points = np.append(points,np.array(newpoint)[None,:],axis=0)

    # print(values.shape,points.shape)
    # print(newpoint,newval)



In [ ]:
x_pred = np.empty((25000,2))
counter = 0
x = np.linspace(0,99,100)
y = np.linspace(0,248,250)

for i in x:
 for j in y:
   x_pred[counter] = np.array([i,j])
   counter += 1

res1 = gp.posterior_mean(x_pred)
res2 = gp.posterior_covariance(x_pred)


X,Y = np.meshgrid(x,y)

PM = np.reshape(res1["f(x)"],(100,250))
PV = np.reshape(res2["v(x)"],(100,250))


fig,ax=plt.subplots(3,1,figsize=(8,6))
ax[0].imshow(PV)
ax[1].imshow(PM)
ax[2].imshow(a[:,2].reshape(100,250))
for aa in ax:
    aa.scatter(points[:-1,1],points[:-1,0],s=1,color='orange')
    aa.scatter(points[-1,1],points[-1,0],s=2,color='red')
    